<a href="https://colab.research.google.com/github/pac-man21/Dynamic-Author-Disambiguation/blob/main/Batch_AD_%2B_INCAD_for_third_data_set(reading_multiple_csv_files).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import glob
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
#!unzip dblp40.zip
path = r'dblp40' # use your path
all_files = glob.glob(path + "/*.csv")
X = []
X_t = []
for filename in all_files:
    df = pd.read_csv(filename)
    df2 = df.sort_values(by = ['year'] , ascending = True )
    X_temp = df2.iloc[ :].values
    Y_temp = df2.iloc[ :].values
    if len(X_temp)  == 0:
      continue
    X.append(X_temp )
    X_t.append(Y_temp )

# Importing Data and finding Frequency of words

In [2]:
n  =  [] # total number of entries
n2 = []
for i in range(0 , len(X ) ):
  n.append(len(X[i] ) )
  n2.append(int(len(X[i])/2 ) )
dnames = {} ## dictionary for just coauthor names  
dorgs = {}## dictionary for organisation names
dkeywords = {} ### dict for keywords
dtopic = {} ### dict for topic keywords
dvenue = {}
dabstract = {}
names = 0 ## total number of coauthor names
orgs = 0  ## total organiasations
keywords = 0 ### total no of keywords
abstract = 0
topic = 0 ##topic keywords
venue = 0
def IPFnames(s ):
  return log2((names+1)/(dnames[s]+1 ) )
def IPFkeywords(s) :
  return log2((keywords+1)/(dkeywords[s]+1 ) )
def IPFtopic(s) :
  return log2((topic+1)/(dtopic[s]+1 ) )
def IPForgs(s):
  return log2((orgs+1)/(dorgs[s]+1 ) )
def IPFvenue(s):
  return log2((venue+1)/(dvenue[s]+1 ) )
def IPFabstract(s):
  return log2( (abstract+1)/(dabstract[s]+1  )  )
sw = stopwords.words('english')
for q in range(0 , len(X ) ):
  mini = 1000000000
  for i in range(0 , n[q] ) :
    X[q][i][2] = X_t[q][i][3]#coauthor
    X[q][i][3] = X_t[q][i][4]#title
    X[q][i][4] = X_t[q][i][6]#keywords
    X[q][i][6] = X_t[q][i][7]#venue
    X[q][i][5] = X_t[q][i][5]#abstract
    X[q][i][7] = X_t[q][i][2]#label
    if int(X[q][i][7] ) <mini:
      mini = int(X[q][i][7] )
  for i in range(0 , n[q] ) :
    X[q][i][7] = int(X[q][i][7] ) - mini
    #X[q][i][8] = int(X_t[q][i][10] )
  #X = Sort(X)
for q in range(0 , len(X ) ):
  for i in range(0 , n[q] ):
    if X[q][i][2]  == X[q][i][2] :

      X[q][i][2] = X[q][i][2].split(';' )
    if X[q][i][3] == X[q][i][3] :
      X[q][i][3] = nltk.word_tokenize(X[q][i][3] )
      X[q][i][3] = {w for w in X[q][i][3] if not w in sw}
      X[q][i][3] = list(nltk.bigrams(X[q][i][3]))
  
    if X[q][i][5] == X[q][i][5] :
      X[q][i][5] = nltk.word_tokenize(X[q][i][5] )
      X[q][i][5] = {w for w in X[q][i][5] if not w in sw}
      X[q][i][5] = list(nltk.bigrams(X[q][i][5]))
    if X[q][i][4] == X[q][i][4] :
      X[q][i][4] = X[q][i][4].split(';')
  for i in range(0 , n[q]):
  
    lis = X[q][i][2]
    if lis!= lis:
      continue
    for s in lis:
      if s in dnames :
        dnames[s]+= 1
      else:
        dnames[s] = 1
      #if dnames[s] >names:
      #  names = dnames[s]
      names+= 1

  for i in range(0 , n[q]) :
    a = X[q][i][3]
    if a!= a:
      continue
    
    a_set = X[q][i][3]
    for words in a_set :
      if words in dtopic:
        dtopic[words ]+= 1
      else:
        dtopic[words] = 1
      #if dtopic[words ] >   topic :
      #  topic = dtopic[words]
      topic+= 1

  for i in range(0 , n[q]):
    a = X[q][i][4]
    if a!= a:
      continue
    lis = X[q][i][4]
    for s in lis:
      if s in dkeywords :
        dkeywords[s]+= 1
      else:
        dkeywords[s] = 1
      #if dkeywords[s] >keywords :
      #  keywords = dkeywords[s]
      keywords+= 1

  for i in range(0 , n[q]):
    lis = X[q][i][5]
    if lis != lis :
      continue
    for s in lis:
     if s in dabstract :
       dabstract[s]+= 1
     else:
       dabstract[s] = 1
     #if abstract < dabstract[s]:
     #  abstract = dabstract[s]
     abstract+= 1
  for i in range(0 , n[q]):
    if X[q][i][6] in dvenue:
      dvenue[X[q][i][6] ]+= 1
    else:
      dvenue[X[q][i][6] ] = 1
    venue+= 1
gold_cluster = []
for i in range(0 , len(X) ) :
  main = []
  key = [ ]
  for j in range(0 ,len(X[i] )  +3 ):
    key.append([] )
  for j in range(0 , len(X[i]) ) :
    key[X[i][j][7]].append(j)
  for i in range(0 , len(key ) ) :
    if len(key[i] ) >0 :
      main.append(key[i] ) 
  gold_cluster.append(main)

**The Similarity Function**

In [3]:
# the function to calculate the similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
def S2( i , j ,  q   ):### this  for author similarity
  num = 0;
  denom = 1;
  v1 = 0
  v2 = 0
  if X[q][i][2]!= X[q][i][2] :
    return 0
  if X[q][j][2]!= X[q][j][2] :
    return 0
  count = [value for value in X[q][i][2] if value in X[q][j][2]]
  for s in X[q][i][2] :
    v1+= IPFnames(s)*IPFnames(s)
   
  for s in X[q][j][2]:
    v2+= IPFnames(s)*IPFnames(s)
  for keys in count:
    num+= IPFnames(keys)*IPFnames(keys)
  if num == 0:
    return 0 
  denom = (v1*v2)**(1/2) ### cube root instead of square root
  return num/denom

def S3(i , j ,  q   ): ### topic similarity using cosine similarity
  num = 0;
  denom = 1;
  val1 = 0
  val2 = 0
  if X[q][i][3]!= X[q][i][3] :
    return 0
  if X[q][j][3]!= X[q][j][3] :
    return 0
  count = [value for value in X[q][i][3] if value in X[q][j][3]] 
  for s in X[q][i][3] :
    val1+= IPFtopic(s)*IPFtopic(s)
   
  for s in X[q][j][3]:
    val2+= IPFtopic(s)*IPFtopic(s)
  
  for keys in count:
      num+= IPFtopic(keys)*IPFtopic(keys)
  if num == 0 :
    return 0 
  #print(str(val1) +  " " + str(val2 ) )
  denom = (val1*val2)**(1/2) ### cube root instead of square root
  return num/denom

def S4( i , j ,  q  ):### keywords comparison
  a = X[q][i][4]
  b = X[q][j][4]
  if a!= a:
    return 0
  if b!= b :
    return 0
  num = 0
  denom = 1
  v1 = 0
  v2 = 0
  count = [value for value in X[q][i][4] if value in X[q][j][4]]
  for s in X[q][i][4] :
    v1+= IPFkeywords(s)*IPFkeywords(s)
   
  for s in X[q][j][4]:
    v2+= IPFkeywords(s)*IPFkeywords(s)

  for keys in count:
    num+= IPFkeywords(keys)*IPFkeywords(keys)
  if num == 0 :
    return 0 
  denom = (v1*v2)**(1/2 ) ## cube root instead of square root(experimenting)
  return num/denom

def S5(i , j , q): ### abstract matching
  a = X[q][i][5]
  b = X[q][j][5]
  if a!= a:
    return 0
  if b!= b :
    return 0
  num =0
  denom = 1
  v1 = 0
  v2 = 0
  count = [value for value in X[q][i][5] if value in X[q][j][5]]
  for s in X[q][i][5] :
    v1+= IPFabstract(s)*IPFabstract(s)
   
  for s in X[q][j][5]:
    v2+= IPFabstract(s)*IPFabstract(s)

  for keys in count:
    num+= IPFabstract(keys)*IPFabstract(keys)
  if num == 0 :
    return 0 
  denom = (v1*v2)**(1/2 ) ## cube root instead of square root(experimenting)
  return num/denom
def S6(i , j , q ): ### venue similarity
  if X[q][i][6] == X[q][j][6] :
    return 1
  else :
    return 0
def S(i , j , v2 , v3 , v4 ,  v6 , v5 ,  c ,  q): ### total similarity 
  #c = 0.05 ### constant assumed for now  , will be replaced by the obtained vector from SVM
  return (S2(i , j  , q )*v2/(v2 + v3 + v4 +  v6 + v5  )+ S3(i , j  , q )*v3/(v2 + v3 + v4 +  v6 + v5  )+ S4(i , j  , q )*v4/(v2 + v3 + v4 +  v6 + v5  ) +S6(i , j , q)*v6/(v2 + v3 + v4 +  v6 + v5  ) + S5(i , j , q)*v5/(v2 + v3 + v4 +  v6 + v5  )   - c) #+S5(i , j)*v5 ### the weighted vector for now is assumed to be one for all , will be replaced after SVM
  #(v2 + v3 + v4 +  v6 + v5  ) 

# Function to minimize(for batch AD )

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
def f(params):
  v2 = params['v2']
  v3 = params['v3']
  v4 = params['v4']
  v5 = params['v5']
  v6 = params['v6']
  #v8 = params['v8' ]
  c = params['c']
  tpn = 0
  tnn = 0
  fpn = 0
  fnn = 0
  val = 0
  for q in range(0 , int(len(X ) )  ):
    A = [] ### this list contains clusters as lists
    U = [] ### all the records initially in the queue
    for i in range(0, int( n2[q] )):
      U.append(i)
    while len(U) >0:
      Q = [] ### this will contain one cluster
      Q.append(U[0])
      
      U.pop(0 )
      temp = []
      i = 0
      for num in  U :
        if  S(Q[0] , num , v2 , v3 , v4 ,  v6  ,v5 , c ,  q )  >0 :
          temp.append(i )
          Q.append(num )
        i+= 1
      for ind in range(len(temp ) -1  , -1 , -1 ) :
        U.pop(temp[ind] )
      A.append(Q) ### adding the current cluster
    lix = []
    for i in range(0 , int( (n2[q] ) )+3):
      lix.append(0)
    for j in range(0 , len(A) ):
      for i in A[j]:
        lix[i] = j;
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for  i in range(0 , int( n2[q] ) ):
      for j in range(i+1 , int( (n2[q] ) )):
        if lix[i] == lix[j]:
          if X[q][i][7] == X[q][j][7]:
            tp+= 1
          else:
            fp+= 1
        else:
          if X[q][i][7] == X[q][j][7]:
            fn+= 1
          else:
            tn+= 1
    tpn+= tp
    tnn+= tn
    fpn+= fp
    fnn += fn
  val = 1 - tpn/(tpn+ (fpn+fnn)/2 )
  return {'loss': val , 'status': STATUS_OK}
fspace = {
    'v2': hp.uniform('v2', 0, 1),
    'v3': hp.uniform('v3', 0, 1),
    'v4': hp.uniform('v4', 0, 1),
    'v5': hp.uniform('v5', 0, 1),
    'v6': hp.uniform('v6', 0, 1),
    #'v8': hp.uniform('v8', 0, 1),
    'c': hp.uniform('c', 0,0.5),
    #'ot2': hp.uniform('ot2', 0, 1),
    #'ot3': hp.uniform('ot3', 0, 1),
    #'ot4': hp.uniform('ot4', 0, 1),
}
trials = Trials()
best = fmin(fn=f, space=fspace, algo=tpe.suggest, max_evals=2000, trials=trials)
print("Best weights for Batch - AD")
print(best )

100%|██████████| 2000/2000 [34:25<00:00,  1.03s/it, best loss: 0.41757285776424535]
Best weights for Batch - AD
{'c': 0.017980037966282468, 'v2': 0.8107459500757824, 'v3': 0.4587640056425373, 'v4': 0.13239274640127788, 'v5': 0.9657554039587471, 'v6': 0.9083709516649476}


# The clustering algorithm

In [ ]:
#best = {'c': 0.17653991385845821, 'ot2': 0.2817655262648923, 'ot3': 0.4190325561855093, 'ot4': 0.9323541973748866, 'v2': 0.7285710784738697, 'v3': 0.8927813727988465, 'v4': 0.8644831252673241, 'v6': 0.6794556512055081, 'v8': 0.667210127338861}
tpn = 0
tnn = 0
fpn = 0
fnn = 0
total_km = 0
import csv
row_list = [["NAME", "True Positive", "True Negative" , "False Positive"  , "False Negative" , "F1-Score" , "Accuracy" , "Precision" , "Recall" , "K-Measure" , "AAP" , "ACP" , "BF1"   ]]
A = [] ### this list contains clusters as lists
bf1_av = 0
for q in range(0 , int(len(X)) ):
  row_liss = []
  row_liss.append(X[q][0][0] )
  A1 = []
  U = [] ### all the records initially in the queue
  for i in range(0 , int(4*n[q]/5)  ) :
    U.append(i)
  while len(U) >0:
    Q = [] ### this will contain one cluster
    Q.append(U[0])
    U.pop(0 )
    temp = []
    i = 0
    for num in  U :
      if  S(Q[0] , num , best['v2'] ,best['v3'] , best['v4'] ,  best['v6'] ,best['v5' ]  , best['c'] ,  q   )  >0 :
        temp.append(i )
        Q.append(num )
      i+= 1
    for ind in range(len(temp ) -1  , -1 , -1 ) :
      U.pop(temp[ind] )
    A1.append(Q) ### adding the current cluster
  A.append(A1)
  lix = []
  for i in range(0 , int(4*n[q]/5)+3):
    lix.append(0)
  for j in range(0 , len(A1) ):
    for i in A1[j]:
      lix[i] = j;
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for  i in range(0 , int(4*n[q]/5)):
    for j in range(i+1 , int(4*n[q]/5) ):
      if lix[i] == lix[j]:
        if X[q][i][7] == X[q][j][7]:
          tp+= 1
        else:
          fp+= 1
      else:
        if X[q][i][7] == X[q][j][7]:
          fn+= 1
        else:
          tn+= 1
  tpn+= tp
  tnn+= tn
  fpn+= fp
  fnn+= fn
  row_liss.append(tp )
  row_liss.append(tn )
  row_liss.append(fp )
  row_liss.append(fn )
  if tp+fp+fn>0 :
    row_liss.append(  (tp)/(tp+ (fp+fn)/2 ) )
  else:
    row_liss.append(0)
  
  row_liss.append(  (tp+tn)/(tp+ tn+ fp+fn ) )

  if tp+fp >0 :
    row_liss.append(tp/(tp+fp ) )
  else:
    row_liss.append(0)

  if tp+fn >0 :
    row_liss.append(tp/(tp+fn ) )
  else:
    row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_list.append(row_liss )
  
aapt = 0
acpt = 0

for q in range(0 , len(X) ) :
  n1 = len(X[q] )
  if n1 == 0:
    continue
  aap = 0
  acp = 0
  for i in range( 0 ,len(gold_cluster[q]) ) :
    l1 = len(gold_cluster[q][i] )
    set_1 = {}
    for elements in gold_cluster[q][i] :
      set_1[elements] = 1
    for clus in A[q]:
      count_1 = 0
      for elemen in clus:
        if elemen in set_1 :
          count_1+= 1
      aap+= (count_1*count_1)/l1

  for i in range( 0 ,len(A[q]) ) :
    l1 = len(A[q][i] )
    set_1 = {}
    for elements in A[q][i] :
      set_1[elements] = 1
    for clus in gold_cluster[q]:
      count_1 = 0
      for elemen in clus:
        if elemen in set_1:
          count_1+= 1
      acp+= (count_1*count_1)/l1
  row_list[q+1][9] = ((aap/len(X[q]) )*(acp/(len(X[q]) ) ) )**0.5
  row_list[q+1][10] = aap/len(X[q] )
  row_list[q+1][11] = acp/len(X[q] )
  row_list[q+1][12] =  (2*(aap/len(X[q]))*(acp/len(X[q] ) ))/( (acp/len(X[q] ) ) +(aap/len(X[q] ) )  )
  total_km+= ((aap/len(X[q]) )*(acp/(len(X[q]) ) ) )**0.5
  aapt+= aap/len(X[q] )
  acpt+= acp/len(X[q] )
  bf1_av+= (2*(aap/len(X[q]))*(acp/len(X[q] ) ))/( (acp/len(X[q] ) ) +(aap/len(X[q] ) )  )
total_km/= len(X)
aapt/= len(X)
acpt/= len(X)
bf1_av/= len(X)
print("True Positive " + str(tpn )  )
print("True Negative " + str(tnn )  )
print("False Positive " + str(fpn )  )
print("False Negative " + str(fnn )  )
print("F1 score for Batch-AD "   +str(tpn/(tpn+ (fpn+fnn)/2 ))   )
print("Accuracy " + str( (tpn+tnn)/(tpn+tnn+fpn+fnn )   )  )

#print("K-Measure " + str(total_km ) )
row_list.append(["NET==>" , tpn , tnn , fpn , fnn ,(tpn)/(tpn+ (fpn+fnn)/2 ) , (tpn+tnn)/(tpn+ tnn+ fpn+fnn ) ,tpn/(tpn+fpn )  , tpn/(tpn+fnn ) ,  total_km  ,aapt , acpt , bf1_av   ] )
with open('Results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)


True Positive 14062
True Negative 98998
False Positive 764
False Negative 25509
F1 score for Batch-AD 0.5170138059084141
Accuracy 0.8114373479362391


# Converting A ==> B

In [ ]:
#sw = stopwords.words('english')
#B = []
#for q in range(0 , len(A) ) :
#  B1 = []
#  for iii in  range(0 , len(A[q])) : 
#    lists = A[q][iii]
#    new = [ ]
#    new.append(lists )
#    dict_names = {}
#    dict_keywords = {}
#    dict_topic = {}
#    dict_org = {}
#    dict_venue = {}
#    tot = 0
#    sum = 0
#    for i in lists :
#      if X[q][i][3] != X[q][i][3]:
#        continue
#      a_set = X[q][i][3]
#      for words in a_set :
#        if words in dict_topic :
#          dict_topic[words ]+= 1
#        else:
#          dict_topic[words] = 1
#  
#    for i in lists:
#      lis = X[q][i ][2]
#      if lis == lis :
#        for s in lis:
#          if s in dict_names:
#            dict_names[s]+= 1
#          else:
#            dict_names[s] = 1
#    
#      lis = X[q][i][4]
#      if lis == lis :
#        for s in lis:
#          if s in dict_keywords:
#            dict_keywords[s]+= 1
#          else:
#            dict_keywords[s] = 1
    #lis = X[i][5]
    #for s in lis:
    #  if s in dict_org:
    #    dict_org[s]+= 1
    #  else:
    #    dict_org[s] = 1
#      s = X[q][i][6]
#      if s == s:
#        if s in dict_venue:
#          dict_venue[s]+= 1
#        else:
#          dict_venue[s] = 1
#      ti = int(X[q][i][8])
#      tot+= 1
#      sum+= t

#    new.append(dict_names )
#    new.append(dict_keywords )
#    new.append(dict_org )
#    new.append(dict_venue )
#    new.append(dict_topic )
#    new.append(tot )
#    new.append(sum )
#    B1.append(new)
#  B.append(B1)

# probability (i , j) function (calculates the probability of record i going in cluster j )

In [ ]:
lm = 0.012
def probability(i , j , xx , v2 , v3 , v4 , v6 ,v5  ,  zz , q , B2) : ### to calculate probability that record i goes to cluster j
  prob = 0
  p2 = 1
  p3 = 1
  p4 = 1
  p5 = 1
  p6 = 1

  s2 = 0
  s3 = 0
  s4 = 0
  s5 = 0
  s6 = 0

  d2 = 0
  d3 = 0
  d4 = 0
  d5 = 0
  d6 = 0
  #print(j )
  for keys in B2[q][j][1] :
    if keys in dnames:
      d2+= IPFnames(keys)*B2[q][j][1][keys]+ IPFnames(keys)
  for keys in B2[q][j][2]:
    if keys in dkeywords:
      d4+= IPFkeywords(keys)*B2[q][j][2][keys]+ IPFkeywords(keys)
    
  for keys in B2[q][j][3]:
    if keys in dabstract:
      d5+= IPFabstract(keys)*B2[q][j][3][keys]+ IPFabstract(keys)
  
  for keys in B2[q][j][4] :
    if keys in dvenue:
      d6+= IPFvenue(keys)*B2[q][j][4][keys]+ IPFvenue(keys)
  
  for keys in B2[q][j][5] :
    if keys in dtopic:
      d3+= IPFtopic(keys )*B2[q][j][5][keys]+ IPFtopic(keys)

  lis = X[q][i][2]
  if lis == lis :
    for s in lis:      
      if s in B2[q][j][1] and s in dnames  :
        p2*= (1 - lm)*(IPFnames(s)*(B2[q][j][1][s]+1) )
        s2+= 1
        if d2 == 0:
          p2 = xx
        else:
          p2/=d2
      else:
        p2*= lm
  a_set = X[q][i][3]
  if a_set == a_set :
    for s in a_set :
      if s in B2[q][j][5] and s in dtopic :
        p3*= (1 - lm)*(IPFtopic(s)*(B2[q][j][5][s]+1) )
        s3+= 1
        if d3 == 0:
          p3 = xx
        else:
          p3/=d3
      else:
        p3*= lm
  
  lis = X[q][i][4]
  if lis == lis :
     for s in lis:      
      if s in B2[q][j][2] and s in dkeywords :
        p4*= (1 - lm)*(IPFkeywords(s)*(B2[q][j][2][s]+1) )
        s4+= 1
        if d4 == 0:
          p4 = xx
        else:
          p4/=d4
      else:
        p4*= lm
  
  lis = X[q][i][5]
  if lis == lis :
    for s in lis:      
      if s in B2[q][j][3] and s in dabstract :
        p5*= (1 - lm)*(IPFabstract(s)*(B2[q][j][3][s]+1) )
        s5+= 1
        if d5 == 0:
          p5 = xx
        else:
          p5/=d5
      else:
        p5*= lm

  lis = X[q][i][6]
  if lis == lis :
    if lis in B2[q][j][4]  and lis in dvenue:
      p6*= (1 - lm)*(IPFvenue(lis)*(B2[q][j][4][lis]+1) )
      s6+= 1
      if   d6 == 0:
        p6 = xx
      else:
        p6 /=d6
    else:
      p6*= lm
  
  if s2 == 0:
    p2 = 0
  if s3 == 0:
    p3 = 0
  if s4 == 0:
    p4 = 0
  if s5 == 0:
    p5 = 0
  if s6 == 0:
    p6 = 0
  prob = (p2*v2/(v2 + v3 + v4 +  v6 + v5  )+p3*v3/(v2 + v3 + v4 +  v6 + v5  )+p4*v4/(v2 + v3 + v4 +  v6 + v5  )   +p6*v6/(v2 + v3 + v4 +  v6 + v5  ) + p5*v5/(v2 + v3 + v4 +  v6 + v5  )  ) - zz    ### rempved organisations for now
  return prob


# Merge Function 

In [ ]:
def merge( i , j , q , B2) :## merging ith and jth cluster


  for ii in B2[q][j][0]:
    B2[q][i][0].append(ii )
  
  for keys in B2[q][j][1]:
    if keys in B2[q][i][1]:
      B2[q][i][1][keys]+= B2[q][j][1][keys]
    else:
      B2[q][i][1][keys] = B2[q][j][1][keys]

  
  for keys in B2[q][j][2]:
    if keys in B2[q][i][2]:
      B2[q][i][2][keys]+= B2[q][j][2][keys]
    else:
      B2[q][i][2][keys] = B2[q][j][2][keys]

  
  for keys in B2[q][j][3]:
    if keys in B2[q][i][3]:
      B2[q][i][3][keys]+= B2[q][j][3][keys]
    else:
      B2[q][i][3][keys] = B2[q][j][3][keys]
    

  for keys in B2[q][j][4]:
    if keys in B2[q][i][4]:
      B2[q][i][4][keys]+= B2[q][j][4][keys]
    else:
      B2[q][i][4][keys] = B2[q][j][4][keys]

  for keys in B2[q][j][5]:
    if keys in B2[q][i][5]:
      B2[q][i][5][keys]+= B2[q][j][5][keys]
    else:
      B2[q][i][5][keys] = B2[q][j][5][keys]


# Add

In [ ]:
def add(i , j , q , B2) : ### add ith record to jth cluster
  lis=  X[q][i][2]
  B2[q][j][0].append(i)
  global names
  global topic
  global keywords
  global orgs
  global venue
  if lis == lis :
    for s in lis:
      if s in B2[q][j][1]:
        B2[q][j][1][s]+= 1
      else:
        B2[q][j][1][s] = 1
    
      #if s in dnames:
      #  dnames[s]+= 1
      #else:
      #  dnames[s] = 1

  

  a_set = X[q][i][3]
  if a_set == a_set :
    for s in a_set :
      if s in B2[q][j][5]:
        B2[q][j][5][s]+= 1
      else:
        B2[q][j][5][s] = 1
      #if s in dtopic :
      #  dtopic[s]+= 1
      #else:
      #  dtopic[s] = 1

  lis=  X[q][i][4]
  if lis == lis :

    for s in lis:
      if s in B2[q][j][2]:
        B2[q][j][2][s]+= 1
      else:
        B2[q][j][2][s] = 1
    
      #if s in dkeywords:
      #  dkeywords[s]+= 1
      #else:
      #  dkeywords[s] = 1
  lis=  X[q][i][5]
  if lis == lis :
    for s in lis:
      if s in B2[q][j][3]:
        B2[q][j][3][s]+= 1
      else:
        B2[q][j][3][s] = 1
  
  lis=  X[q][i][6]
  if lis == lis :
    for s in lis:
      venue+= 1
      if s in B2[q][j][4]:
        B2[q][j][4][s]+= 1
      else:
        B2[q][j][4][s] = 1
    
      #if s in dvenue:
      #  dvenue[s]+= 1
      #else:
      #  dvenue[s] = 1
  

# f1 score for Inc-AD

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
def f2(params):
  sw = stopwords.words('english')
  B = []
  AA = A.copy()
  for q in range(0 , len(AA) ) :
    B1 = []
    for iii in  range(0 , len(AA[q])) : 
      lists = AA[q][iii].copy()
      new = [ ]
      new.append(lists )
      dict_names = {}
      dict_keywords = {}
      dict_topic = {}
      dict_abs = {}
      dict_venue = {}
      tot = 0
      sum = 0
      for i in lists :
        if X[q][i][3] != X[q][i][3]:
          continue
        a_set = X[q][i][3].copy()
        for words in a_set :
          if words in dict_topic :
            dict_topic[words ]+= 1
          else:
            dict_topic[words] = 1

      for i in lists:
        lis = X[q][i ][2]
        if lis == lis :
          for s in lis:
            if s in dict_names:
              dict_names[s]+= 1
            else:
              dict_names[s] = 1
    
        lis = X[q][i][4]
        if lis == lis :
          for s in lis:
            if s in dict_keywords:
              dict_keywords[s]+= 1
            else:
              dict_keywords[s] = 1
        lis = X[q][i][5]
        if lis == lis :
          for s in lis:
            if s in dict_abs:
              dict_abs[s]+= 1
            else:
              dict_abs[s] = 1
        s = X[q][i][6]
        if s == s:
          if s in dict_venue:
            dict_venue[s]+= 1
          else:
            dict_venue[s] = 1

      new.append(dict_names )
      new.append(dict_keywords )
      new.append(dict_abs )
      new.append(dict_venue )
      new.append(dict_topic )
      B1.append(new)
    B.append(B1)
  v2 = params['v2']
  v3 = params['v3']
  v4 = params['v4']
  v6 = params['v6']
  v5 = params['v5']
  xx = params['xx']
  zz = params['zz']
  tpn = 0
  tnn = 0
  fpn = 0
  fnn = 0
  val = 0
  for q in range(0 , len(B) ) :
    for i in range(int(4*n[q]/5  ) ,  n[q] ) :
      indices = []
      for j in range(0 , len(B[q]) ):
        w = probability(i , j , xx , v2 , v3 , v4 , v6 , v5 ,  zz , q , B )
        if w >0 :
          indices.append(j)
      if len(indices) == 1 :
        add(i , indices[0] , q , B )
      elif len(indices) == 0:
        new = [ ]
        liss = []
        liss.append(i)
        new.append(liss )
        dict_names = {}
        dict_keywords = {}
        dict_topic = {}
        dict_abs = {}
        dict_venue = {}
        for j in liss :
          a_set = X[q][j][3]
          if a_set == a_set :
            for words in a_set :
              if words in dict_topic :
                dict_topic[words ]+= 1
              else:
                dict_topic[words] = 1
  
        for j in liss:
          lis = X[q][j ][2]
          if X[q][j][2] == X[q][j][2] :
            for s in lis:
              if s in dict_names:
                dict_names[s]+= 1
              else:
                dict_names[s] = 1
          lis = X[q][j][4]
          if lis == lis :
            for s in lis:
              if s in dict_keywords:
                dict_keywords[s]+= 1
              else:
                dict_keywords[s] = 1

          lis = X[q][j][5]
          if lis == lis :
            for s in lis:
              if s in dict_abs:
                dict_abs[s]+= 1
              else:
                dict_abs[s] = 1
        
          s = X[q][j][6]
          if s == s :
            if s in dict_venue:
              dict_venue[s]+= 1
            else:
              dict_venue[s] = 1

        new.append(dict_names )
        new.append(dict_keywords )
        new.append(dict_abs )
        new.append(dict_venue )
        new.append(dict_topic )
        B[q].append(new)
      else:
        indices.sort()
        for j in range(1 , len(indices )):
          merge(indices[0] , indices[j] , q , B )
        add(i , indices[0] , q  , B)
        for j in range(len(indices ) -1 , 0 , -1 ) :
          B[q].pop(j )
    lix = []
    for i in range(0 , n[q] +3):
      lix.append(0)
    for j in range(0 , len(B[q]) ):
      for i in B[q][j][0]:
        lix[i] = j;
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for  i in range(0 , n[q]):
      for j in range(i+1 , n[q] ):
        if lix[i] == lix[j]:
          if X[q][i][7] == X[q][j][7]:
            tp+= 1
          else:
            fp+= 1
        else:
          if X[q][i][7] == X[q][j][7]:
            fn+= 1
          else:
            tn+= 1
    tpn+= tp
    tnn+= tn
    fpn+= fp
    fnn+= fn
  val = 1 - tpn/(tpn+ (fpn+fnn)/2 )
  return {'loss': val, 'status': STATUS_OK}
fspace2 = {
    'v2': hp.uniform('v2', 0, 1),
    'v3': hp.uniform('v3', 0, 1),
    'v4': hp.uniform('v4', 0, 1),
    'v6': hp.uniform('v6', 0, 1),
    'v5': hp.uniform('v5', 0, 1),
    'xx': hp.uniform('xx', 0, 1),
    'zz': hp.uniform('zz', 0, 1)
}
trials = Trials()
best = fmin(fn=f2, space=fspace2, algo=tpe.suggest, max_evals=100, trials=trials)
print("The best weights for INC-AD are :" )
print(best )

100%|██████████| 100/100 [05:09<00:00,  3.09s/it, best loss: 0.5212806135789199]
The best weights for INC-AD are :
{'v2': 0.8297582902670521, 'v3': 0.39560842216427095, 'v4': 0.9321092042822331, 'v5': 0.9669160309210715, 'v6': 0.8123504263765522, 'xx': 0.39135205522301164, 'zz': 0.015090050965799073}


# Incremental AD part


In [ ]:

import csv
#best = {'v2': 0.28425313104946415, 'v3': 0.34291573859926106, 'v4': 0.6202389767140876, 'v6': 0.42362875766082364, 'v8': 0.00755099434643261, 'xx': 0.6197730140281698, 'zz': 0.0052293082585595976}
#best = {'v2': 0.09482086579888616, 'v3': 0.8471501871839383, 'v4': 0.227375127224119, 'v6': 0.7610259640103285, 'v8': 0.037882362335591724, 'xx': 0.09252278668498393, 'zz': 0.0987549851564282}
#best = {'v2': 0.7692034188054258, 'v3': 0.7709283498000158, 'v4': 0.4238832884883059, 'v6': 0.6222362791388469, 'v8': 0.05021103828322537, 'xx': 0.3106629561483645, 'zz': 0.2001225224042803}
#best = {'v2': 0.3884545474682226, 'v3': 0.6898329199214819, 'v4': 0.42808303929029634, 'v6': 0.21694771508646665, 'v8': 0.9554908439616692, 'xx': 0.33192205484399906, 'zz': 0.06295177074731975}
sw = stopwords.words('english')
B = []
for q in range(0 , len(A) ) :
  B1 = []
  for iii in  range(0 , len(A[q])) : 
    lists = A[q][iii]
    new = [ ]
    new.append(lists )
    dict_names = {}
    dict_keywords = {}
    dict_topic = {}
    dict_abs = {}
    dict_venue = {}
    tot = 0
    sum = 0
    for i in lists :
      if X[q][i][3] != X[q][i][3]:
        continue
      a_set = X[q][i][3]
      for words in a_set :
        if words in dict_topic :
          dict_topic[words ]+= 1
        else:
          dict_topic[words] = 1
  
    for i in lists:
      lis = X[q][i ][2]
      if lis == lis :
        for s in lis:
          if s in dict_names:
            dict_names[s]+= 1
          else:
            dict_names[s] = 1
    
      lis = X[q][i][4]
      if lis == lis :
        for s in lis:
          if s in dict_keywords:
            dict_keywords[s]+= 1
          else:
            dict_keywords[s] = 1
      lis = X[q][i][5]
      if lis == lis :
        for s in lis:
          if s in dict_abs:
            dict_abs[s]+= 1
          else:
            dict_abs[s] = 1
      s = X[q][i][6]
      if s == s:
        if s in dict_venue:
          dict_venue[s]+= 1
        else:
          dict_venue[s] = 1


    new.append(dict_names )
    new.append(dict_keywords )
    new.append(dict_abs )
    new.append(dict_venue )
    new.append(dict_topic )
    new.append(tot )
    new.append(sum )
    B1.append(new)
  B.append(B1)
row_list = [["NAME", "True Positive", "True Negative" , "False Positive"  , "False Negative" , "F1-Score" , "Accuracy" , "Precision" , "Recall" ,"AAP" , "ACP"  , "K-Measure"  , "BF1"  ]]
tpn = 0
tnn = 0
fpn = 0
fnn = 0
total_km = 0
bf1_av = 0 
for q in range(0 , len(B) ) :
  row_liss = []

  row_liss.append(X[q][0][0] )
  for i in range(int(4*n[q]/5  ) , n[q] ) :
    indices = []

    for j in range(0 , len(B[q]) ):
      w = probability(i , j , best['xx'] ,best['v2'] ,best['v3'] ,best['v4'] ,best['v6'] ,best['v5' ]  , best['zz'] , q  , B )
      if w >0 :
        indices.append(j)
    if len(indices) == 1 :
      add(i , indices[0] , q , B )
    elif len(indices) == 0:
      new = [ ]
      liss = []
      liss.append(i)
      new.append(liss )
      dict_names = {}
      dict_keywords = {}
      dict_topic = {}
      dict_abs = {}
      dict_venue = {}
      for j in liss :
        a_set = X[q][j][3]
        for words in a_set :
          if words in dict_topic :
            dict_topic[words ]+= 1
          else:
            dict_topic[words] = 1
        #if words in dtopic :
        #  dtopic[words ]+= 1
        #else:
        #  dtopic[words] = 1
  
      for j in liss:
        lis = X[q][j ][2]
        if lis == lis :
          for s in lis:
            if s in dict_names:
              dict_names[s]+= 1
            else:
              dict_names[s] = 1
        #if s in dnames:
        #  dnames[s]+= 1
        #else:
        #  dnames[s] = 1
        lis = X[q][j][4]
        if lis == lis :
          for s in lis:
            if s in dict_keywords:
              dict_keywords[s]+= 1
            else:
              dict_keywords[s] = 1
        lis = X[q][j][5]
        if lis == lis :
          for s in lis:
            if s in dict_abs:
              dict_abs[s]+= 1
            else:
              dict_abs[s] = 1
            
        s = X[q][j][6]
        if s in dict_venue:
          dict_venue[s]+= 1
        else:
          dict_venue[s] = 1
      #if s in dvenue:
      #  dvenue[s]+= 1
      #else:
      #  dvenue[s] = 1
      new.append(dict_names )
      new.append(dict_keywords )
      new.append(dict_abs )
      new.append(dict_venue )
      new.append(dict_topic )
      B[q].append(new)
    else:
      indices.sort()
      for j in range(1 , len(indices )):
        merge(indices[0] , indices[j] , q , B )
      add(i , indices[0] , q  , B )
      for j in range(len(indices ) -1 , 0 , -1 ) :
        B[q].pop(j )

  lix = []
  for i in range(0 , n[q]+3):
    lix.append(0)
  for j in range(0 , len(B[q]) ):
    for i in B[q][j][0]:
      lix[i] = j;
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  for  i in range(0 , n[q]):
    for j in range(i+1 , n[q] ):
      if lix[i] == lix[j]:
        if X[q][i][7] == X[q][j][7]:
          tp+= 1
        else:
          fp+= 1
      else:
        if X[q][i][7] == X[q][j][7]:
          fn+= 1
        else:
          tn+= 1
  tpn+= tp
  tnn+= tn
  fpn+= fp
  fnn += fn

  row_liss.append(tp )
  row_liss.append(tn )
  row_liss.append(fp )
  row_liss.append(fn )
  
  if tp+fp+fn>0 :
    row_liss.append(  (tp)/(tp+ (fp+fn)/2 ) )
  else:
    row_liss.append(0)
  row_liss.append(  (tp+tn)/(tp+ tn+ fp+fn ) )

  if tp+fp >0 :
    row_liss.append(tp/(tp+fp ) )
  else:
    row_liss.append(0)

  if tp+fn >0 :
    row_liss.append(tp/(tp+fn ) )
  else:
    row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_liss.append(0)
  row_list.append(row_liss )
aapt = 0
acpt = 0
for q in range(0 , len(X) ) :
  n1 = len(X[q] )
  
  aap = 0
  acp = 0
  for i in range( 0 ,len(gold_cluster[q]) ) :
    l1 = len(gold_cluster[q][i] )
    set_1 = {}
    for clus in B[q]:
      for elements in gold_cluster[q][i] :
        set_1[elements] = 1
      count_1 = 0
      for elemen in clus[0]:
        if elemen in set_1 :
          if set_1[elemen] == 1:
            count_1+= 1
            set_1[elemen] = 0
      aap+= (count_1*count_1)/l1

  for i in range( 0 ,len(B[q]) ) :
    l1 = len(B[q][i][0] )
    set_1 = {}
    for clus in gold_cluster[q]:
      for elements in B[q][i][0] :
        set_1[elements] = 1
      count_1 = 0
      for elemen in clus:
        if elemen in set_1:
          count_1+= 1
          set_1[elemen ] = 0
      acp+= (count_1*count_1)/l1
  row_list[q+1][9] = (aap/len(X[q]))
  row_list[q+1][10] = (acp/len(X[q] ) )
  row_list[q+1][11] = ((aap/len(X[q]) )*(acp/(len(X[q]) ) ) )**0.5
  row_list[q+1][12] = (2*(aap/len(X[q]))*(acp/len(X[q] ) ))/( (acp/len(X[q] ) ) +(aap/len(X[q] ) )  )
  total_km+= ((aap/len(X[q]) )*(acp/(len(X[q]) ) ) )**0.5
  aapt+= (aap/len(X[q]))
  acpt+= (acp/len(X[q]))
  bf1_av+= (2*(aap/len(X[q]))*(acp/len(X[q] ) ))/( (acp/len(X[q] ) ) +(aap/len(X[q] ) )  )


total_km/= len(X)
aapt/= len(X)
acpt/= len(X)
bf1_av/= len(X)
print("True Positive " + str(tpn )  )
print("True Negative " + str(tnn )  )
print("False Positive " + str(fpn )  )
print("False Negative " + str(fnn )  )
print("F1 score for INC-AD "   +str(tpn/(tpn+ (fpn+fnn)/2 ))   )
print("Accuracy " + str( (tpn+tnn)/(tpn+tnn+fpn+fnn )   )  )
print(total_km )
row_list.append(["NET==>" , tpn , tnn , fpn , fnn ,(tpn)/(tpn+ (fpn+fnn)/2 ) , (tpn+tnn)/(tpn+ tnn+ fpn+fnn ) ,tpn/(tpn+fpn )  , tpn/(tpn+fnn ) ,aapt , acpt ,  total_km  , bf1_av   ] )
with open('Results2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

True Positive 32956
True Negative 114625
False Positive 41902
False Negative 29870
F1 score for INC-AD 0.47871938642108014
Accuracy 0.6728013749527018
0.4516367472103229
